<a href="https://colab.research.google.com/github/MWFK/Nike_Case_Assignments/blob/main/VIP_promotion_Python_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectives

**Build** a small data pipeline that outputs **an overview of Nike’s VIP customers**, **ordered by the total sales value of their purchases**. 

---

For this promotion, the marketing department is only interested in **VIPs currently located in The Netherlands.** 


---


**VIPs that have not purchased any products are still eligible** – they are VIP, after all. 


---


This ask is not a one-off, as Marketing is already thinking about running this promotion again next year if it is successful.


---


In the overview, Marketing is looking for:

  The name of the VIP

  The email address of the VIP

  The total sales value of the VIPs purchases

# Load Libs

In [36]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# Import Data

In [52]:
# Data source: Sales domain
# One month's data

path_transactions = '/content/transactions.parquet'
df_transactions   = pq.read_table(path_transactions)
df_transactions   = df_transactions.to_pandas()

# print(df_transactions.shape)
# print(df_transactions.dtypes)
# df_transactions

In [53]:
# Data source: Customer domain
# Two week's data
# Date format yyyy-mm-dd

path_vips_1101 = '/content/vips_2020-11-01.csv'
df_vips_1101   = pd.read_csv (path_vips_1101)

# print(df_vips_1101.shape)
# print(df_vips_1101.dtypes)
# df_vips_1101.head()

In [54]:
# Data source: Customer domain
# One week's data

path_vips_1115 = '/content/vips_2020-11-15.csv'
df_vips_1115   = pd.read_csv (path_vips_1115)

# print(df_vips_1115.shape)
# print(df_vips_1115.dtypes)
# df_vips_1115.head()

In [55]:
# Data source: Customer domain
# One week's data

path_vips_1125 = '/content/vips_2020-11-25.csv'
df_vips_1125   = pd.read_csv (path_vips_1125)

# print(df_vips_1125.shape)
# print(df_vips_1125.dtypes)
# df_vips_1125.head()

In [56]:
# Source: Marketing domain
# One month's  data
# Date format dd-mm-yyyy
'''
'meta_change_date' have happened in 04-01-2020 and 25-01-2020, 
VIP's data have occured on 01-11-2020, 15-11-2020, 25-11-2020, 
Which means all the VIP data files needs to be updated according 
to he mapping file, before using them. 
'''

path_vips_mapping = '/content/umd_vip_to_profile_mapping.csv'
df_vips_mapping = pd.read_csv(path_vips_mapping, sep=';')

# print(df_vips_mapping.shape)
# print(df_vips_mapping.dtypes)
# df_vips_mapping.head()

# Overview of Nike’s VIP customers (2020-11-01) 

1. **vips_2020-11-01.csv** is linked to **umd_vip_to_profile_mapping.csv** through the column **vip_id**

        We'll use it to add **umd_vip_to_profile_mapping.csv** columns to **vips_2020-11-01.csv** columns

2. **vips_2020-11-01.csv** is linked to **transactions.parquet** through the column **profile_id**

        We'll use it to add **transactions.parquet** columns to **vips_2020-11-01.csv** columns

### [1] VIP's data filtering, cleansing and merging

In [43]:
# Only keep VIP's currently based in the Netherlands
df_vips_1101 = df_vips_1101.loc[df_vips_1101['country'] == 'The Netherlands'].reset_index(drop=True)
df_vips_1101

,vip_id,first_name,last_name,vip_type,country,email
0,4544056563,Brandon,Ritter,Athlete,The Netherlands,Brandon.Ritter@nike.com
1,7531485682,Edwin,Floyd,NaN,The Netherlands,Edwin.Floyd@nike.com
2,4271215441,Kristin,Brooks,Donator,The Netherlands,Kristin@Brooks.com


In [44]:
''' 
An athlete got his status active as no since he was duplicated in the system
and since from the meta_commets we have no other indication, that means he still an active VIP, 
so we'll delete that duplicate VIP ID with active status no
'''

df_vips_mapping.drop_duplicates(subset='vip_id', keep="first", inplace=True)
df_vips_mapping.head()

,vip_id,profile_id,active,meta_change_date,meta_comments
0,4544056563,1-5151-4642-1,yes,NaN,NaN
2,9339003839,1-4786-9297-9,yes,NaN,NaN
3,1600627714,1-4947-2509-6,yes,NaN,NaN
4,7531485682,1-57006-547-0,no,25-1-2020,No longer a donator
5,4271215441,1-9860-3438-0,yes,NaN,NaN


In [45]:
# Merge df_vips_mapping with df_vips_1101 using 'vip_id'

df_vips_1101_mapped = pd.merge(df_vips_1101, df_vips_mapping, on='vip_id', how='left').reset_index(drop=True)
df_vips_1101_mapped

,vip_id,first_name,last_name,vip_type,country,email,profile_id,active,meta_change_date,meta_comments
0,4544056563,Brandon,Ritter,Athlete,The Netherlands,Brandon.Ritter@nike.com,1-5151-4642-1,yes,NaN,NaN
1,7531485682,Edwin,Floyd,NaN,The Netherlands,Edwin.Floyd@nike.com,1-57006-547-0,no,25-1-2020,No longer a donator
2,4271215441,Kristin,Brooks,Donator,The Netherlands,Kristin@Brooks.com,1-9860-3438-0,yes,NaN,NaN


In [46]:
# Only keep active VIP's

df_vips_1101_mapped = df_vips_1101_mapped.loc[df_vips_1101_mapped['active'] == 'yes'].reset_index(drop=True)
df_vips_1101_mapped

,vip_id,first_name,last_name,vip_type,country,email,profile_id,active,meta_change_date,meta_comments
0,4544056563,Brandon,Ritter,Athlete,The Netherlands,Brandon.Ritter@nike.com,1-5151-4642-1,yes,NaN,NaN
1,4271215441,Kristin,Brooks,Donator,The Netherlands,Kristin@Brooks.com,1-9860-3438-0,yes,NaN,NaN


In [47]:
cols = ['profile_id', 'first_name', 'email']
df_vips_1101_mapped = df_vips_1101_mapped[cols]
df_vips_1101_mapped

,profile_id,first_name,email
0,1-5151-4642-1,Brandon,Brandon.Ritter@nike.com
1,1-9860-3438-0,Kristin,Kristin@Brooks.com


### [2] Transactions filtering, cleansing and aggregation 

In [48]:
# Filter transactions dates(between 2020-11-01 and 2020-11-14) 
df_transactions_1101 = df_transactions[(df_transactions['transaction_date'] >= '2020-11-01') & (df_transactions['transaction_date'] < '2020-11-15')]

# Drop the cancelled items
df_transactions_1101 = df_transactions_1101.loc[df_transactions_1101['cancellation_flag'] != 'yes'].reset_index(drop=True)

'''
discount_amount_per_unit values should be prepapred to calculate the unit price after discount so if it's:
NaN it should be changed with the value 1.
100.0 should be 0.
The rest should be divided by 100.
'''
df_transactions_1101['discount_amount_per_unit'] = np.where(df_transactions_1101['discount_amount_per_unit'] == 100, 0, df_transactions_1101['discount_amount_per_unit'])
# df_transactions_1101['discount_amount_per_unit'] = df_transactions_1101['discount_amount_per_unit'].fillna(1)
df_transactions_1101['discount_amount_per_unit'] = np.where(df_transactions_1101['discount_amount_per_unit'] >= 0, df_transactions_1101['discount_amount_per_unit']/100, 1)

# Calculate the unit price after applying the discount
df_transactions_1101['unit_price_after_discount'] = df_transactions_1101['recommended_retail_price_per_unit'] * df_transactions_1101['discount_amount_per_unit'] * df_transactions_1101['quantity']
df_transactions_1101

# Filter out unecessary columns
cols_trans = ['profile_id', 'unit_price_after_discount']
df_transactions_1101 = df_transactions_1101[cols_trans]
df_transactions_1101

,profile_id,unit_price_after_discount
0,1-4947-2509-6,678.0
1,1-4947-2509-6,678.0
2,1-4947-2509-6,800.0
3,1-4786-9297-9,2694.0
4,1-9860-3438-0,362.0
5,1-9860-3438-0,0.0
6,0-89923-575-1,782.0


In [49]:
df_transactions_1101 = df_transactions_1101.groupby(['profile_id']).sum()
df_transactions_1101

,unit_price_after_discount
profile_id,
0-89923-575-1,782.0
1-4786-9297-9,2694.0
1-4947-2509-6,2156.0
1-9860-3438-0,362.0


### [3] Final result

In [51]:
# Merge df_transactions_1101 with df_vips_1101_mapped

df_vips_1101_mapped_transactions = pd.merge(df_vips_1101_mapped, df_transactions_1101, on='profile_id', how='left').reset_index(drop=True)
df_vips_1101_mapped_transactions = df_vips_1101_mapped_transactions[['first_name', 'email', 'unit_price_after_discount']]
df_vips_1101_mapped_transactions

,first_name,email,unit_price_after_discount
0,Brandon,Brandon.Ritter@nike.com,NaN
1,Kristin,Kristin@Brooks.com,362.0


In [57]:
'''
Interpretations
Brandon did not make a purchase during the first two weeks.
Kristin have bought two of the same unit during the first two weeks, 
Then he returned one of them during the last week 
Therefore, we should keep the minus values as an indications for returned units.
'''

'\nInterpretations\nBrandon did not make a purchase during the first two weeks.\nKristin have bought two of the same unit during the first two weeks, \nThen he returned one of them during the last week \nTherefore, we should keep the minus values as an indications for returned units.\n'